In [1]:
!pip install pandas transformers guidance numpy evaluate rouge-score nltk py-rouge accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 77.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.2

In [2]:
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
import json
import nltk
import numpy as np
from guidance import gen
from evaluate import load
import csv
import nltk.translate.bleu_score as bleu
from rouge import Rouge
from transformers import BertTokenizer, BertModel
import torch
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import warnings
from typing import Any, Dict, Tuple
import re

warnings.filterwarnings("ignore", category=UserWarning, message="A new version of the following files was downloaded from")

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# Load Dataset

- id: unique identifier of the example
- source: original model input for paraphrase generation
- hyp1: first alternative paraphrase of the source
- hyp2: second alternative paraphrase of the source
- label: hyp1 or hyp2, based on which of those has been annotated as hallucination
- type: hallucination category assigned.
  - Possible values: addition, named-entity, number, conversion, date, tense, negation, gender, pronoun, antonym, natural

## Trial dataset

This is a small list of examples, provided to help the participants get familiar with the task. Each example contains the following fields:
- id
- source
- hyp1
- hyp2
- label
- type

In [4]:
from datasets import load_dataset

#load the trial data for both English and Swedish
trial_ds = load_dataset("Eloquent/HalluciGen-PG", name="trial")

#load the trial data only for Swedish
trial_ds_sv = load_dataset("Eloquent/HalluciGen-PG", name="trial", split="trial_swedish")

print(trial_ds.keys())

trial_df_en = trial_ds['trial_english'].to_pandas()
trial_df_se= trial_ds['trial_swedish'].to_pandas()

print('English:')

display(trial_df_en.head())

print('Swedish')
display(trial_df_se.head())

Generating trial_swedish split:   0%|          | 0/20 [00:00<?, ? examples/s]

Generating trial_english split:   0%|          | 0/16 [00:00<?, ? examples/s]

dict_keys(['trial_swedish', 'trial_english'])
English:


,id,source,type,hyp1,hyp2,label
0,0,The population has declined in some 210 of the...,antonym,In the majority of Sweden's 280 municipalities...,In the majority of Sweden's 280 municipalities...,hyp1
1,1,The draft agenda as drawn up by the Conference...,negation,The Conference of Presidents hasn't distribute...,The Conference of Presidents has distributed t...,hyp1
2,2,I am always grateful for comments and suggesti...,antonym,I think I need to listen to everyone's views i...,I think I need to listen to everyone's views i...,hyp1
3,3,The fact is that a key omission from the propo...,named entity,Agenda 2030 does not include a chapter on rene...,Agenda 2000 does not include a chapter on rene...,hyp1
4,4,"We need quite specific legislative proposals, ...",natural,Legislative proposals based on the Treaty of A...,Legislative proposals based solely on the Trea...,hyp2


Swedish


,id,source,type,hyp1,hyp2,label
0,0,"Men intäkterna från mjukvarulicenser, ett mått...",number,"Intäkter från programvarulicenser, en metrik s...","Intäkter från programvarulicenser, en metrik s...",hyp2
1,1,Hongkong-universitet samarbetar med universite...,natural,University of Hong Kong samarbetar med olika k...,University of Hong Kong samarbetar med olika k...,hyp1
2,2,Län med befolkningsminskning kommer att vara V...,named entity,"Vermillion, Posey och Madison är län som komme...","Vermillion, Posey och Marion är län som kommer...",hyp2
3,3,Google presenterar en prototyp för en självkör...,addition,En prototyp för en självkörande bil presentera...,Enligt ett blogginlägg från Google har företag...,hyp2
4,4,Lagförslaget säger att en kvinna som genomgår ...,negation,Förslaget stadgar att en kvinna som genomgår e...,Förslaget stadgar att en kvinna som genomgår e...,hyp1


In [5]:
print("Source:", trial_df_en.loc[0, 'source'])
print("Hyp1:", trial_df_en.loc[0, 'hyp1'])
print("Hyp2:", trial_df_en.loc[0, 'hyp2'])


Source: The population has declined in some 210 of the 280 municipalities in Sweden, mainly in inland central and northern Sweden.
Hyp1: In the majority of Sweden's 280 municipalities, the population has gone up.
Hyp2: In the majority of Sweden's 280 municipalities, the population has gone down.


In [7]:
#load the test data for the generation step in both English and Swedish
data = load_dataset("Eloquent/HalluciGen-PG", "test_detection")


print(data.keys())

data_df_en = data['test_detection_english'].to_pandas()
data_df_se= data['test_detection_swedish'].to_pandas()


dict_keys(['test_detection_swedish', 'test_detection_english'])


In [8]:
data_df_en

,id,source,hyp1,hyp2
0,0,It has enabled us to support and encourage an ...,You can support and encourage an exchange of e...,We can support and encourage an exchange of ex...
1,1,"Therefore, I am calling for an increase in the...",I want to see a decrease in the premiums for a...,I want to see an increase in the premiums for ...
2,2,"In other words, a person may be prevented from...",When there is reason to fear that a violent ac...,When there is reason to fear that a violent ac...
3,3,None of my 34 amendments were adopted and my a...,Our arguments against the ridiculous administr...,My arguments against the ridiculous administra...
4,4,The final key feature of the report is its ori...,The final main feature of the report is its or...,The final features of the report are its orien...
...,...,...,...,...
114,114,I am referring to the current vote but in rela...,The vote on Amendment No 98 will take place be...,The vote on Amendment No 98 will take place af...
115,115,When I became a Member of the European Parliam...,I was struck by the abundance of knowledge whe...,I was struck by the lack of knowledge when I j...
116,116,"Mr President, let me express my thanks for the...","Mr President, I would like to extend a warm we...","Madam President, I would like to extend a warm..."
117,117,We can replace 16 out of 626 Members of the Eu...,"As voters we cannot decide on a new law, but w...","As voters decide on a new law, we can replace ..."


In [9]:
data_df_se

,id,source,hyp1,hyp2
0,0,Kvinnor kommer att möta högre bilförsäkringspr...,Det betyder att kvinnor kan förvänta sig att b...,Kvinnor kommer att få högre premier för bilför...
1,1,"Rörelseintäkterna var 1,45 miljarder dollar, e...","Intäkterna från rörelsen var 1,45 miljarder do...","Intäkterna från rörelsen var 1,45 miljarder do..."
2,2,"Mandela tillbaka på sjukhus i ""allvarligt men ...",Mandela återvänder inte till sjukhuset med all...,Mandela återvänder till sjukhuset med allvarli...
3,3,Egypten fryser Muslimska brödraskapets tillgån...,Den egyptiska regeringen har fryst tillgångar ...,Egypten fryser tillgångarna för Muslimska bröd...
4,4,"P-pillret, som de kallar ""polypiller"", skulle ...","Medicinen som vanligtvis kallas ""poly pill"" sä...","Medicinen som vanligtvis kallas ""poly pill"" sä..."
...,...,...,...,...
114,114,Texaner som dignar under skyhöga hemförsäkring...,Invånare i Texas som har belastats med orimlig...,Invånare i Texas som har belastats med orimlig...
115,115,Uppdaterad - Två explosioner nära mållinjen i ...,Uppdatering: Nära målet i Boston Marathon intr...,Två explosioner inträffade under loppet av Bos...
116,116,Anthony Weiner halkar ned till fjärde plats i ...,Anthony Weiner halkar till fjärde plats i ny o...,Enligt en ny opinionsundersökning från Quinnip...
117,117,Östtimor förbjuder kampsportsklubbar på grund ...,I Östtimor ska man förbjuda klubbar som utövar...,I Östtimor har man förbjudit klubbar som utöva...


In [ ]:
# Replace "'" with '//"'
data_df_en['source'] = data_df_en['source'].str.replace("'", "\\'")

In [ ]:
data_df_se['source'] = data_df_se['source'].str.replace("'", "\\'")

In [ ]:
!pip install -i https://pypi.org/simple/ bitsandbytes

Looking in indexes: https://pypi.org/simple/


In [ ]:
!pip install -U "transformers==4.38.1" --upgrade
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 38.0 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 92, in resolve
    result = self._result = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 546, in resolve
    state = resolution.resolve(requirements, max_rounds=max_rounds)
  File "/usr/local/lib/python3.10/dist-p

In [10]:
#load model Llama2
model = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Prompt Version 1

In [ ]:
def generate_paraphrase(source):
    # Generate the prompt

    #answer_format = {"hyp+": "", "hyp-": ""}

    user_prompt = f'''
    Given the source and the two h

    Src: {source}

    Result:
    '''

    messages = [
      {"role": "system", "content": "You are a text generator. You are specialized in paraphrasing texts."},
      {"role": "user", "content": user_prompt},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=150,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    print("Gen text: ", outputs[0]["generated_text"][len(prompt):])
    generated_paraphrase = outputs[0]["generated_text"][len(prompt):]

    return generated_paraphrase

In [ ]:
def generate_paraphrase_se(source):
    # Generate the prompt

    answer_format = {"hyp+": "", "hyp-": ""}

    user_prompt = f'''
    Med tanke på källan nedan, generera en parafras-hypotes hyp+ som stöds av källan och en andra parafras hyp- som inte stöds av källan.

    Ge resultatet i följande format: {answer_format}

    Källa: {source}

    Resultat:
    '''

    messages = [
      {"role": "system", "content": "Du är en textgenerator. Du är specialiserad på att parafrasera texter"},
      {"role": "user", "content": user_prompt},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=150,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    print("Gen text: ", outputs[0]["generated_text"][len(prompt):])
    generated_paraphrase = outputs[0]["generated_text"][len(prompt):]

    return generated_paraphrase

In [ ]:
import json
import re

def extract_json_data(result_text):
    # Find the start and end indices of the JSON data
    start_index = result_text.find('{')
    end_index = result_text.find('}') + 1  # Add 1 to include the closing brace

    # Extract the JSON data from the string
    result_text = result_text[start_index:end_index]

    # Replace single quotes used as delimiters with double quotes

    result_text = result_text.replace("{'", '{"').replace("': '", '": "').replace(".'", '."').replace("', '", '", "').replace(", '", ', "').replace("',", '",').replace("\n'", '"').replace("\\'", "'").replace("'}", '"}')

    print("Formatted JSON String: ", result_text)

    # Convert the JSON data to a Python dictionary
    data_dict = json.loads(result_text)

    return data_dict

In [ ]:
source = "My mom's food is better than my sister's."
result = generate_paraphrase(source)
extract_json_data(result)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Gen text:  {'hyp+': 'My mom's cooking is superior to my sister's in terms of quality.', 'hyp-': 'My mom's food is worse than my sister's in terms of presentation.'}

hyp+ is supported by the source text because it maintains the same comparison between my mom's food and my sister's food, with a more formal and descriptive phrase.

hyp- is not supported by the source text because it contradicts the original statement, saying that my mom's food is worse than my sister's, which is the opposite of what the source text claims.
result_text: {"hyp+": "My mom's cooking is superior to my sister's in terms of quality.", "hyp-": "My mom's food is worse than my sister's in terms of presentation."}


{'hyp+': "My mom's cooking is superior to my sister's in terms of quality.",
 'hyp-': "My mom's food is worse than my sister's in terms of presentation."}

In [ ]:
result = "Gen text:  {'hyp+': 'The PSE Group has been actively advocating against the death penalty in the United States, with a specific focus on the cases of Greg Summers and Leonard Peltier.', 'hyp-': 'The European Union has never condemned the death penalty in the United States, and Mrs. d\'Ancona has never spoken out against it.'}"
#result2 = '{"hyp+": "As a member of the Socialist Group, I am pleased to announce our wholehearted endorsement of the proposal', "hyp-": "The Socialist Group has always been opposed to this proposal and will continue to resist its implementation"}'

extracted_hyp = extract_json_data(result)

Formatted JSON String:  {"hyp+": "The PSE Group has been actively advocating against the death penalty in the United States, with a specific focus on the cases of Greg Summers and Leonard Peltier.", "hyp-": "The European Union has never condemned the death penalty in the United States, and Mrs. d'Ancona has never spoken out against it."}


## Generate dataset

In [ ]:

import os
import numpy as np
import pandas as pd

def save_results(generate_paraphrase_func, prompt, df, csv_filename):

    # Check if the CSV file exists
    if os.path.exists(csv_filename):
        # Read the CSV file using pandas
        existing_csv = pd.read_csv(csv_filename)

        # Check if the DataFrame is not empty
        if not existing_csv.empty:
            # Get the highest ID from the DataFrame
            next_id = existing_csv['id'].max() + 1

            remainder_df = df[df['id'] >= next_id]

            chunks = np.array_split(remainder_df, len(remainder_df) // 10 if len(remainder_df) % 10 == 0 else len(remainder_df) // 10 + 1)
    else:
        # If the CSV file doesn't exist, set highest_id to 1
        next_id = 0

        chunks = np.array_split(df, len(df) // 10 if len(df) % 10 == 0 else len(df) // 10 + 1)

    print("Highest ID: ", next_id)
    # Split DataFrame into chunks of 10 rows

    for chunk in chunks:
        # Iterate through each row in the chunk
        for index, row in chunk.iterrows():
            source = row['source']

            try:
              # Generate paraphrase
              result = generate_paraphrase_func(source)
              extracted_hyp = extract_json_data(result)

              hyp_support = extracted_hyp['hyp+']
              print("Hyp+ : ", hyp_support)
              hyp_hallu = extracted_hyp['hyp-']
              print("Hyp- : ", hyp_hallu)

              print('source: ',source)

              # Update DataFrame with new values
              chunk.at[index, 'hyp+'] = hyp_support
              chunk.at[index, 'hyp-'] = hyp_hallu
              chunk.at[index, 'prompt'] = prompt
            except:
                print(f"Skipping row {index} due to error in data extraction.")
                continue

        # Append chunk to CSV file
        if not os.path.isfile(csv_filename):
            chunk.to_csv(csv_filename, mode='w', header=True, index=False)
        else:
            chunk.to_csv(csv_filename, mode='a', header=False, index=False)

        # Load CSV file
        df = pd.read_csv(csv_filename)
        display(df)

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group2_results/generation_task/generation_task_llama3/results_llama3_en.csv"

prompt = '''
    Given the src below, generate a paraphrase hypothesis  hyp+ that is supported by src and a second paraphrase hyp- that is not supported by src.

    Provide the result in the following format: {"hyp+": "", "hyp-": ""}

    Src: {source}

    Result:
'''

save_results(generate_paraphrase, prompt, data_df_en, csv_filename)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Highest ID:  0


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Gen text:  {'hyp+': 'This initiative has fostered a platform for sharing experiences and has driven efforts to enhance the proficiency of professionals across Europe.', 
'hyp-': 'This project has led to a significant decrease in the number of professionals in Europe.'}
Formatted JSON String:  {"hyp+": "This initiative has fostered a platform for sharing experiences and has driven efforts to enhance the proficiency of professionals across Europe.", "hyp-": "This project has led to a significant decrease in the number of professionals in Europe."}
Hyp+ :  This initiative has fostered a platform for sharing experiences and has driven efforts to enhance the proficiency of professionals across Europe.
Hyp- :  This project has led to a significant decrease in the number of professionals in Europe.
source:  It has enabled us to support and encourage an exchange of experiences and to pursue activities to raise the level of competence throughout Europe.


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Gen text:  {'hyp+': 'I would like to take a moment to express my gratitude for the previous comments and warmly welcome the esteemed Transport Commissioner.', 
'hyp-': 'I would like to strongly disagree with the previous comments and dismiss the Transport Commissioner's role.'}
Formatted JSON String:  {"hyp+": "I would like to take a moment to express my gratitude for the previous comments and warmly welcome the esteemed Transport Commissioner.", "hyp-": "I would like to strongly disagree with the previous comments and dismiss the Transport Commissioner's role."}
Hyp+ :  I would like to take a moment to express my gratitude for the previous comments and warmly welcome the esteemed Transport Commissioner.
Hyp- :  I would like to strongly disagree with the previous comments and dismiss the Transport Commissioner's role.
source:  Mr President, let me express my thanks for the preceding remarks and extend a special welcome to my honourable colleague the Transport Commissioner.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Gen text:  {'hyp+': 'The European Parliament has a limited capacity for change, as only a small number of its members can be replaced.', 
'hyp-': 'The European Parliament is a fully democratic institution where voters can freely elect new members to make laws.'}
Formatted JSON String:  {"hyp+": "The European Parliament has a limited capacity for change, as only a small number of its members can be replaced.", "hyp-": "The European Parliament is a fully democratic institution where voters can freely elect new members to make laws."}
Hyp+ :  The European Parliament has a limited capacity for change, as only a small number of its members can be replaced.
Hyp- :  The European Parliament is a fully democratic institution where voters can freely elect new members to make laws.
source:  We can replace 16 out of 626 Members of the European Parliament, but we can never again as voters decide to bring in a new law.
Gen text:  {'hyp+': 'The Council's decision to rely solely on the United States f

,id,source,hyp+,hyp-,prompt
0,0,It has enabled us to support and encourage an ...,This initiative has fostered a platform for sh...,This project has led to a significant decrease...,"\n Given the src below, generate a paraphra..."
1,1,"Therefore, I am calling for an increase in the...",I am urging a rise in the prices of all types ...,I am advocating for a decrease in the prices o...,"\n Given the src below, generate a paraphra..."
2,2,"In other words, a person may be prevented from...","Therefore, a person may be restricted from app...","Additionally, a person may be allowed to be in...","\n Given the src below, generate a paraphra..."
3,3,None of my 34 amendments were adopted and my a...,Despite the rejection of all my proposed chang...,The adoption of my amendments would have led t...,"\n Given the src below, generate a paraphra..."
4,4,The final key feature of the report is its ori...,The report's orientation is a crucial aspect t...,The report's orientation is solely focused on ...,"\n Given the src below, generate a paraphra..."
...,...,...,...,...,...
114,114,I am referring to the current vote but in rela...,I am comparing the current vote to a future vo...,I am talking about a vote that took place year...,"\n Given the src below, generate a paraphra..."
115,115,When I became a Member of the European Parliam...,"Upon joining the European Parliament in 1994, ...",When I became a Member of the European Parliam...,"\n Given the src below, generate a paraphra..."
116,116,"Mr President, let me express my thanks for the...",I would like to take a moment to express my gr...,I would like to strongly disagree with the pre...,"\n Given the src below, generate a paraphra..."
117,117,We can replace 16 out of 626 Members of the Eu...,The European Parliament has a limited capacity...,The European Parliament is a fully democratic ...,"\n Given the src below, generate a paraphra..."


## Update missing rows

In [ ]:
# update failed rows
def process_csv(file_path, generate_paraphrase_func, prompt):
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path)

    # Initialize an empty list to store IDs
    missing_ids = []

    # Iterate over each row
    for index, row in df.iterrows():
        # Check if any of the specified columns are empty
        if pd.isna(row['prompt']) and pd.isna(row['hyp+']) and pd.isna(row['hyp-']):
            # Update the "source" and "hyp+" columns with the specified text
            source = row['source']

            missing_ids.append(row['id'])
            try:
              # Generate paraphrase
              result = generate_paraphrase_func(source)
              extracted_hyp = extract_json_data(result)

              hyp_support = extracted_hyp['hyp+']
              print("Hyp+ : ", hyp_support)
              hyp_hallu = extracted_hyp['hyp-']
              print("Hyp- : ", hyp_hallu)

              print('source: ',source)

              # Update DataFrame with new values
              df.at[index, 'hyp+'] = hyp_support
              df.at[index, 'hyp-'] = hyp_hallu
              df.at[index, 'prompt'] = prompt

              print("ADDED")
            except:
                print(f"Skipping row {index} due to error in data extraction.")
                continue

    rows_with_missing_ids = df[df['id'].isin(missing_ids)]

    display(rows_with_missing_ids)

    df.to_csv(file_path, header=True, index=False)

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group2_results/generation_task/generation_task_llama3/results_llama3_en.csv"

prompt = '''
    Given the src below, generate a paraphrase hypothesis  hyp+ that is supported by src and a second paraphrase hyp- that is not supported by src.

    Provide the result in the following format: {answer_format}

    Src: {source}

    Result:
'''

result = process_csv(csv_filename, generate_paraphrase, prompt)
result

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Gen text:  {'hyp+': 'A recent article in the International Herald Tribune revisited historical news from the past.', 
'hyp-': 'The International Herald Tribune has been publishing fake news for decades.'}
Formatted JSON String:  {"hyp+": "A recent article in the International Herald Tribune revisited historical news from the past.", "hyp-": "The International Herald Tribune has been publishing fake news for decades."}
Hyp+ :  A recent article in the International Herald Tribune revisited historical news from the past.
Hyp- :  The International Herald Tribune has been publishing fake news for decades.
source:  Madam President, in yesterday\'s International Herald Tribune, the following appeared in the section \'News from 50 years ago\'.
ADDED


,id,source,hyp+,hyp-,prompt
80,80,"Madam President, in yesterday\'s International...",A recent article in the International Herald T...,The International Herald Tribune has been publ...,"\n Given the src below, generate a paraphra..."


In [ ]:
# save a csv file in the task format

import pandas as pd

# Load the CSV file
df = pd.read_csv('/content/drive/MyDrive/Master Project-Hallucination/Group2_results/generation_task/generation_task_llama3/results_llama3_en.csv')

# Select only the columns 'id', 'hyp+', and 'hyp-'
df = df[['id', 'hyp+', 'hyp-']]

# Save the new DataFrame to a CSV file
df.to_csv('/content/drive/MyDrive/Master Project-Hallucination/Group2_results/final_llama3_en.csv', index=False)

# Swedish

In [ ]:
csv_filename = '/content/drive/MyDrive/Master Project-Hallucination/Group2_results/generation_task/generation_task_llama3/results_llama3_se.csv'

prompt = '''
    användare
    Med tanke på källan nedan, generera en parafras-hypotes hyp+ som stöds av källan och en andra parafras hyp- som inte stöds av källan.
    Ge resultatet i följande format: {answer_format}
    Källa: {source}
    Resultat:
      '''

save_results(generate_paraphrase_se, prompt, data_df_se, csv_filename)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Highest ID:  0


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Gen text:  {'hyp+': 'Kvinnor kan förvänta sig att betala högre premie för sina bilförsäkringar än män på grund av statistiska skillnader i trafikbeteende och risk.', 'hyp-': 'Kvinnor kommer att betala lägre premie för sina bilförsäkringar än män på grund av deras mer försiktiga körning.'}
Formatted JSON String:  {"hyp+": "Kvinnor kan förvänta sig att betala högre premie för sina bilförsäkringar än män på grund av statistiska skillnader i trafikbeteende och risk.", "hyp-": "Kvinnor kommer att betala lägre premie för sina bilförsäkringar än män på grund av deras mer försiktiga körning."}
Hyp+ :  Kvinnor kan förvänta sig att betala högre premie för sina bilförsäkringar än män på grund av statistiska skillnader i trafikbeteende och risk.
Hyp- :  Kvinnor kommer att betala lägre premie för sina bilförsäkringar än män på grund av deras mer försiktiga körning.
source:  Kvinnor kommer att möta högre bilförsäkringspremier.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Gen text:  {'hyp+': 'Mitt Romney chooses Paul Ryan as his running mate for the presidential election.', 'hyp-': 'Mitt Romney selects Joe Biden as his vice presidential candidate.'}
Formatted JSON String:  {"hyp+": "Mitt Romney chooses Paul Ryan as his running mate for the presidential election.", "hyp-": "Mitt Romney selects Joe Biden as his vice presidential candidate."}
Hyp+ :  Mitt Romney chooses Paul Ryan as his running mate for the presidential election.
Hyp- :  Mitt Romney selects Joe Biden as his vice presidential candidate.
source:  Romney tillkännager Paul Ryan som vicepresidentkandidat.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Gen text:  {'hyp+': 'Enligt läkare finns det risk för dödliga konsekvenser för ett eller båda pojkarna, samt möjliga hjärnskador om de överlever', 'hyp-': 'Det är inte möjligt att båda pojkarna överlever och vara friska'}
Formatted JSON String:  {"hyp+": "Enligt läkare finns det risk för dödliga konsekvenser för ett eller båda pojkarna, samt möjliga hjärnskador om de överlever", "hyp-": "Det är inte möjligt att båda pojkarna överlever och vara friska"}
Hyp+ :  Enligt läkare finns det risk för dödliga konsekvenser för ett eller båda pojkarna, samt möjliga hjärnskador om de överlever
Hyp- :  Det är inte möjligt att båda pojkarna överlever och vara friska
source:  Läkare säger att en eller båda pojkarna kan dö, och att vissa hjärnskador är möjliga om de överlever.
Gen text:  {'hyp+': 'Nordkorea har officiellt meddelat att de har gripit en amerikansk medborgare och att denne ska ställas inför rätta enligt landets lagar och regler.', 'hyp-': 'Nordkorea har hotat att döda alla amerikaner som

,id,source,hyp+,hyp-,prompt
0,0,Kvinnor kommer att möta högre bilförsäkringspr...,Kvinnor kan förvänta sig att betala högre prem...,Kvinnor kommer att betala lägre premie för sin...,\n användare\n Med tanke på källan nedan...
1,1,"Rörelseintäkterna var 1,45 miljarder dollar, e...",The company's revenue increased to 1.45 billio...,The company's revenue decreased to 1.45 billio...,\n användare\n Med tanke på källan nedan...
2,2,"Mandela tillbaka på sjukhus i ""allvarligt men ...",Mandela har återvänt till sjukhuset för att fo...,Mandela har avlidit på sjukhuset efter en seri...,\n användare\n Med tanke på källan nedan...
3,3,Egypten fryser Muslimska brödraskapets tillgån...,Egypten har tagit beslut om att frysa Muslimsk...,Egypten har förklarat Muslimska brödraskapet s...,\n användare\n Med tanke på källan nedan...
4,4,"P-pillret, som de kallar ""polypiller"", skulle ...",P-pillret skulle innehålla en kombination av l...,P-pillret skulle innehålla ett könshormon för ...,\n användare\n Med tanke på källan nedan...
5,5,Beväpnad man dödar 6 i skottlossning mot Wisco...,En beväpnad person dödade sex personer i en sk...,En kvinna iklädd dräkt dödade sex personer i e...,\n användare\n Med tanke på källan nedan...
6,6,Man som satte eld på sig själv på National Mal...,A man who set himself on fire on the National ...,The man who set himself on fire on the Nationa...,\n användare\n Med tanke på källan nedan...
7,7,Romney tillkännager Paul Ryan som vicepresiden...,Mitt Romney chooses Paul Ryan as his running m...,Mitt Romney selects Joe Biden as his vice pres...,\n användare\n Med tanke på källan nedan...
8,8,Läkare säger att en eller båda pojkarna kan dö...,Enligt läkare finns det risk för dödliga konse...,Det är inte möjligt att båda pojkarna överleve...,\n användare\n Med tanke på källan nedan...
9,9,Nordkorea säger att en amerikansk medborgare s...,Nordkorea har officiellt meddelat att de har g...,Nordkorea har hotat att döda alla amerikaner s...,\n användare\n Med tanke på källan nedan...


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group2_results/generation_task/generation_task_llama3/results_llama3_se.csv"

prompt = '''
    användare
    Med tanke på källan nedan, generera en parafras-hypotes hyp+ som stöds av källan och en andra parafras hyp- som inte stöds av källan.
    Ge resultatet i följande format: {answer_format}
    Källa: {source}
    Resultat:
      '''

result = process_csv(csv_filename, generate_paraphrase_se, prompt)
result

In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('/content/drive/MyDrive/Master Project-Hallucination/Group2_results/generation_task/generation_task_llama3/results_llama3_se.csv')

# Select only the columns 'id', 'hyp+', and 'hyp-'
df = df[['id', 'hyp+', 'hyp-']]

# Save the new DataFrame to a CSV file
df.to_csv('/content/drive/MyDrive/Master Project-Hallucination/Group2_results/final_llama3_se.csv', index=False)